In [11]:
%matplotlib auto
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import sys
import os.path
sys.path.insert(0, os.getcwd()+"/..")
from SEMContour import *
sys.path.insert(0, os.getcwd()+"/../../common")
from FileUtil import gpfs2WinPath

CWD = '/gpfs/WW/BD/MXP/SHARED/SEM_IMAGE/IMEC/Case02_calaveras_v3/3Tmp/CT_KPI_test/Calaveras_v3_regular_CT_KPI_003_slope_modified_revert_all_patterns/h/cache/dummydb/result/MXP/job1/ContourSelectModelCalibration430result1'
#CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\ContourSelectModelCalibration430result1'
#CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob1\h\cache\dummydb\result\MXP\job1\ContourExtraction400result1'
CWD = gpfs2WinPath(CWD)
allNeighborColNames = ['NeighborContinuity', 'NeighborOrientation', 'NeighborParalism']

class ContourAnalyzer(object):
    """docstring for ContourData"""
    def __init__(self, contourfile):
        self.__build(contourfile)

    def __build(self, contourfile):
        contour = SEMContour()
        contour.parseFile(contourfile)
        if not contour:
            sys.exit("ERROR: read in contour file %s fails\n" % contourfile)
        self.contour = contour
        self.df = contour.toDf()
# get contour data
patternid = '2438'
contourfile = os.path.join(CWD, patternid+'_image_contour.txt')
ca = ContourAnalyzer(contourfile)
df = ca.df

Using matplotlib backend: Qt5Agg


In [1]:
print('hello')

hello


In [15]:
# get contour data
contourfile = os.path.join(CWD, '461_image_contour.txt')
ca = ContourAnalyzer(contourfile)
df = ca.df

In [8]:
# plot the SEM contour and angle
def plot_contour_angle(ca, patternid='', arrow_length=1):
    df = ca.df
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    imw, imh = ca.contour.getshape()
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    ax.set_title("Pattern "+patternid+ " image Contour")
    
    # plot image
    
    # plot contour
    ax.plot(df.loc[:, 'offsetx'], df.loc[:, 'offsety'], 'b.')
    
    # plot angle
    for _, row in df.iterrows():
        x, y = row.loc['offsetx'], row.loc['offsety']
        angle = row.loc['angle']
        dx, dy = arrow_length*np.cos(angle), arrow_length*np.sin(angle)
        ax.arrow(x, y, dx, dy, width=0.1, fc='y', ec='y') # ,shape='right', overhang=0
        
    plt.gca().invert_yaxis()
    plt.show()
plot_contour_angle(ca, '461')

In [18]:
# plot the histgram for the modified slope, & plot by filter
print(df.columns)
colname = 'NeighborOrientation' # NeighborParalism
df[colname].plot.hist(bins=100)
def plot_col_filter(ca, patternid='', colname=''):
    df = ca.df
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    imw, imh = ca.contour.getshape()
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    ax.set_title("Pattern "+patternid)
    
    thresh = 0.99
    flt_gt = df.loc[:, colname] > thresh
    flt_eq = df.loc[:, colname] == thresh
    flt_lt = df.loc[:, colname] < thresh
    
    ax.plot(df.loc[flt_gt, 'offsetx'], df.loc[flt_gt, 'offsety'], 'b.', markersize=2, label=colname+'>{}'.format(thresh))
    ax.plot(df.loc[flt_eq, 'offsetx'], df.loc[flt_eq, 'offsety'], 'r.', markersize=2, label=colname+'=={}'.format(thresh))
    ax.plot(df.loc[flt_lt, 'offsetx'], df.loc[flt_lt, 'offsety'], 'r.', markersize=2, label=colname+'<{}'.format(thresh))

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()
plot_col_filter(ca, patternid='461', colname=colname)

Index([u'polygonId', u'offsetx', u'offsety', u'angle', u'weight',
       u'confidence', u'intensity', u'slope', u'band_width',
       u'ridge_intensity', u'curvature', u'contrast', u'mxp_flag',
       u'EigenRatio', u'NeighborOrientation', u'NeighborParalism',
       u'ClfLabel'],
      dtype='object')


In [4]:
# plot by column unique labels
def plot_col_by_label(ca, patternid='', colname=''):
    contour = ca.contour
    df = ca.df
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid)
    
    uniqVals = df.loc[:, colname].drop_duplicates().values
    print(uniqVals)
    for label in uniqVals:
        flt_eq = df.loc[:, colname] == label
        if label == 'nan':
            flt_eq = df.loc[:, colname].isna()
        ax.plot(df.loc[flt_eq, 'offsetx'], df.loc[flt_eq, 'offsety'], '.', linestyle='None',  markersize=2, label=colname+'=={}'.format(label))

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [ ]:
# plot contour filtering by ridge_intensity 
def plot_rd_filter(ca, patternid=''):
    df = ca.df
    imw, imh = ca.contour.getshape()
    
    figw = 9
    fig = plt.figure(figsize=(figw, figw))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])

    plt.gca().invert_yaxis()
    thresh = 0.003
    flt = df.loc[:, 'ridge_intensity'] > thresh
    ax.plot(df.loc[flt, 'offsetx'], df.loc[flt, 'offsety'], 'g.', markersize=2, label='ridge_intensity>{}'.format(thresh))
    ax.plot(df.loc[~flt, 'offsetx'], df.loc[~flt, 'offsety'], 'r.', markersize=3, label='ridge_intensity<={}'.format(thresh))
    
    ax.set_title(patternid+" Rg>{} filter".format(thresh))
    plt.legend()
    plt.show()
plot_rd_filter(ca, 'Pattern 3658')

In [ ]:
# for pop out plot
%matplotlib qt5

def plot_reg(ca, winname=''):
    df = ca.df
    imw, imh = ca.contour.getshape()
    
    colstr = 'slope  ridge_intensity'
    cols = colstr.split()
    #df = df[cols]
    df.loc[:, 'slope'] = df.loc[:, 'slope'].abs().values
    ## df = df.loc[df.slope<0.03, :]
    x, y = df.loc[:, 'slope'], df.loc[:, 'ridge_intensity']

    # from scipy import stats
    # slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    #plt.plot(x, intercept + slope*x, 'r', label='fitted ridge_intensity')
    import statsmodels.api as sm
    X = sm.add_constant(x, prepend=False)
    results = sm.OLS(y, X).fit()
    # print(results.summary())
    # print(results.mse_resid, results.mse_total)
    # print(results.params, type(results.params))
    k, b = results.params.loc['slope'], results.params.loc['const']
    
    from statsmodels.sandbox.regression.predstd import wls_prediction_std
    pred_std, predict_ci_low, predict_ci_upp = wls_prediction_std(results)

    xmax, ymax = x.max(), y.max()
    figw = 7
    fig = plt.figure(figsize=(figw, figw*ymax/xmax))
    #fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlim([0, xmax*1.1])
    ax.set_ylim([0, ymax*1.1])
    
    ax.set_title(winname+' ridge_intensity v.s. abs of modified slope')
    ax.set_xlabel('modified slope')
    ax.set_ylabel('ridge_intensity')
    
    ax.plot(x, y, 'o', label='original ridge_intensity v.s. slope')
    y_pred = results.predict()
    ax.plot(x, y_pred, 'r', label='predicted ridge_intensity={:.3f}slope+{:.3f}, $R^2={:.3f}$'.format(k, b, results.rsquared))
    plt.plot(x, predict_ci_low, 'b--', lw=1, label='predict lower')
    plt.plot(x, predict_ci_upp, 'g--', lw=1, label='predict upper')
    
    df.loc[:, 'predict_ci_low'] = predict_ci_low
    df.loc[:, 'predict_ci_upp'] = predict_ci_upp
    
    plt.legend()
    #plt.show()
    
    ## ridge_intensity prediction boundary plot
    figw = 9
    fig = plt.figure(figsize=(2*figw, figw))
    ax = fig.add_subplot(1, 2, 1)
    #fig = plt.figure(2)
    #ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    plt.gca().invert_yaxis()
    #ax.set_ylim(ax.get_ylim()[::-1])
    
    # print(df.columns)
    flt = (df.ridge_intensity>=df.predict_ci_low)& (df.ridge_intensity<=df.predict_ci_upp)
    nonzero = df.slope != 0
    #ax.plot(df.loc[flt ,'offsetx'], 1024-1-df.loc[flt, 'offsety'], 'b.', markersize=3, label='ridge_intensity In prediction range')
    ax.plot(df.loc[flt ,'offsetx'], df.loc[flt, 'offsety'], 'b.', markersize=2, label='ridge_intensity In prediction range')
    ax.plot(df.loc[(~flt)&nonzero ,'offsetx'], df.loc[(~flt)&nonzero, 'offsety'], 'co', markersize=5, label='Rg Out prediction range, slope!=0')
    ax.plot(df.loc[(~flt)&(~nonzero) ,'offsetx'], df.loc[(~flt)&(~nonzero), 'offsety'], 'rd', markersize=5, label='Rg Out prediction range, slope==0')
    ax.set_title(winname+" Rg outside prediction range of $Rg={:.3f}slope+{:.3f}$".format(k, b))
    ax.legend()
    
    ## ridge_intensity > thresh filter plot
    ax = fig.add_subplot(1, 2, 2)
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])

    plt.gca().invert_yaxis()
    thresh = 0.003
    flt = df.loc[:, 'ridge_intensity'] > thresh
    ax.plot(df.loc[flt, 'offsetx'], df.loc[flt, 'offsety'], 'g.', markersize=2, label='ridge_intensity>{}'.format(thresh))
    ax.plot(df.loc[~flt, 'offsetx'], df.loc[~flt, 'offsety'], 'r.', markersize=3, label='ridge_intensity<={}'.format(thresh))
    
    ax.set_title(winname+" Rg>{} filter".format(thresh))
    ax.legend()
    
    
    plt.show()
    
    #resid=y-y_pred
    #rss=np.sum(resid**2)
    #MSE=np.sqrt(rss/(result.nobs-2))
    
    def ols_quantile(m, X, q):
      # m: Statsmodels OLS model.
      # X: X matrix of data to predict.
      # q: Quantile.
      #
      from scipy.stats import norm
      mean_pred = m.predict(X)
      se = np.sqrt(m.scale)
      return mean_pred + norm.ppf(q) * se
    
    #print(ols_quantile(results, X, 0.5))
    return results

    
results = plot_reg(ca)
print("ridge_intensity v.s. slope regression results:")
print(results.summary())
print('\nresults.mse\n', results.mse_resid, results.mse_total, '\n')
print("results.params\n", results.params, type(results.params))

In [ ]:
patterns = [1, 444, 461, 1001, 3658]
contourfiles= [CWD+'/{}_image_contour.txt'.format(pid) for pid in patterns]


- for slightly better for visualazation

    %matplotlib notebook 

- normal

    %matplotlib inline 

In [ ]:
%matplotlib qt5
### ridge_intensity v.s. slope regression plot for more patterns
for ix, contourfile in enumerate(contourfiles):
    ca = ContourAnalyzer(contourfile)
    iminfo = 'Pattern '+str(patterns[ix])
    plot_reg(ca, iminfo)
    #plot_rd_filter(ca, iminfo)

In [ ]:
%matplotlib auto
def plot_reg2(df):
    colstr = 'slope  ridge_intensity'
    cols = colstr.split()
    df = df[cols]
    df.loc[:, 'slope'] = df.loc[:, 'slope'].abs().values
    x, y = df.loc[:, 'slope'], df.loc[:, 'ridge_intensity']

    from scipy import stats
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, y, 'o', label='original '+colstr)
    ax.plot(x, intercept + slope*x, 'r', label='ridge_intensity={:.2f}slope+{:.2f}'.format(slope, intercept))
    ax.set_xlim([0, x.max()*1.1])
    ax.set_ylabel(r"ridge_intensity")
    ax.set_xlabel("slope")
    plt.legend()
    plt.show()
plot_reg2(ca.df)

In [16]:
def addNeighborFeatures(df):
    '''
    add Features for the input contour DataFrame, based on the neighbor relationship in the context of segment

    Parameters:
    -----------
    df: [in, out] contour as DataFrame
        [in] Contour df, must contains `polygonId`, `angle`, `offsetx`, `offsety`
        [out] Contour df, added `NeighborContinuity`, `NeighborOrientation`, `NeighborParalism`

            - `NeighborContinuity`:  |X(n) - X(n-1)|^2, usually is to 1 (because of 8-neighbor contour tracing)
            - `NeighborOrientation`:  dot(EigenVector(n), EigenVector(n-1)), closer to 1, the better(may use 1-dot)
            - `NeighborParalism`:  ||cross((X(n) - X(n-1)), EigenVector(n-1))||, closer to 1, the better(may use 1-cross)
    TODO, the segment neighborhood based features can only be obtained by the whole segment, can't use ROI cropped segment 
    '''
    if len(df) <= 0:
        return df
    polygonIds = df.loc[:, 'polygonId'].drop_duplicates().values
    preIdx = df.index[0]
    for polygonId in polygonIds:
        isPolygonHead = True
        for curIdx, _ in df.loc[df['polygonId']==polygonId, :].iterrows():
            NeighborContinuity = 1
            NeighborOrientation = 1
            NeighborParalism = 1
            if not isPolygonHead:
                eigenvector_n_1 = np.array([np.cos(df.loc[preIdx, 'angle']), np.sin(df.loc[preIdx, 'angle'])])
                eigenvector_n = np.array([np.cos(df.loc[curIdx, 'angle']), np.sin(df.loc[curIdx, 'angle'])])
                neighorvector = np.array([df.loc[curIdx, 'offsetx'] - df.loc[preIdx, 'offsetx'],
                                        df.loc[curIdx, 'offsety'] - df.loc[preIdx, 'offsety']])
                crossvector = np.cross(neighorvector, eigenvector_n_1)

                NeighborContinuity = np.sqrt(neighorvector.dot(neighorvector))
                NeighborOrientation = eigenvector_n.dot(eigenvector_n_1)
                NeighborParalism = np.sqrt(crossvector.dot(crossvector))/NeighborContinuity
                NeighborContinuity = NeighborContinuity
            preIdx = curIdx
            isPolygonHead = False

            for ii, val in enumerate([NeighborContinuity, NeighborOrientation, NeighborParalism]):
                colname = allNeighborColNames[ii]
                df.loc[curIdx, colname] = val
    return df

def plot_multi_filters(ca, patternid='', strFlts=None, transform_filter=False):
    if strFlts is None:
        newStrFlts = {}
    elif not isinstance(strFlts, dict):
        newStrFlts = {}
        for col in allNeighborColNames:
            for strFlt in strFlts:
                if col in strFlt:
                    newStrFlts[col] = strFlt
                    break
    strFlts = newStrFlts
    print(strFlts)
    df = ca.df
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    xini, yini, xend, yend = ca.contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid)
    
    # plot contour
    ax.plot(df.loc[:, 'offsetx'], df.loc[:, 'offsety'], 'k.', markersize=1, label='SEM Contour')
    # plot angle
    for _, row in df.iterrows():
        x, y = row.loc['offsetx'], row.loc['offsety']
        angle = row.loc['angle']
        arrow_length = 1
        dx, dy = arrow_length*np.cos(angle), arrow_length*np.sin(angle)
        ax.arrow(x, y, dx, dy, width=0.1, fc='y', ec='y') # ,shape='right', overhang=0

    # plot filters
    if not transform_filter:
        for strFlt in strFlts.values():
            curdf = df.query(strFlt)
            ax.plot(curdf.loc[:, 'offsetx'], curdf.loc[:, 'offsety'], 'o', markersize=4, label=strFlt, alpha=0.6)
    else:
        '''
        inflection_points = []
        for strFlt in strFlts.values():
            curdf = df.query(strFlt )
            print(strFlt, len(curdf))
            if len(curdf) != 0:
                inflection_points.append(curdf)
        inflection_df = pd.concat(inflection_points)
        '''
        inflection_df = df.query('or '.join(strFlts.values()))
        print(inflection_df[['polygonId', 'offsetx', 'offsety'] + allNeighborColNames[1:]])
        
        polygonIds = inflection_df.loc[:, 'polygonId'].drop_duplicates().values
        for polygonId in polygonIds:
            curdf = inflection_df.loc[inflection_df['polygonId']==polygonId, :]
            maxNeighborContinuity = curdf.max()['NeighborContinuity']
            minNeighborContinuity, minNeighborOrientation, minNeighborParalism = curdf.min()[allNeighborColNames]

            if minNeighborParalism < minNeighborOrientation and len(curdf.query(strFlts['NeighborParalism'])) > 0:
                idxmin = curdf['NeighborParalism'].idxmin()
                ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'ro', markersize=4, label='minNeighborParalism', alpha=0.6)
            elif minNeighborOrientation < minNeighborParalism and len(curdf.query(strFlts['NeighborOrientation'])) > 0:
                idxmin = curdf['NeighborOrientation'].idxmin()
                ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'rd', markersize=4, label='minNeighborOrientation', alpha=0.6)
            elif len(curdf.query(strFlts['NeighborContinuity'])) > 0:
                if abs(maxNeighborContinuity-1) > abs(minNeighborContinuity-1):
                    idxmax = curdf['NeighborContinuity'].idxmax()
                    ax.plot(curdf.loc[idxmax, 'offsetx'], curdf.loc[idxmax, 'offsety'], 'bd', markersize=4, label='maxNeighborContinuity', alpha=0.6)
                else:
                    idxmin = curdf['NeighborContinuity'].idxmin()
                    ax.plot(curdf.loc[idxmin, 'offsetx'], curdf.loc[idxmin, 'offsety'], 'bo', markersize=4, label='minNeighborContinuity', alpha=0.6)

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()
    
    #inflection_df.plot.scatter(x=allNeighborColNames[1], y=allNeighborColNames[2])
    #plt.show()

df = addNeighborFeatures(df)
ca.df = df
plot_multi_filters(ca, patternid=patternid, strFlts=['NeighborParalism<0.98', 'NeighborOrientation<0.98'], transform_filter=False)

{'NeighborParalism': 'NeighborParalism<0.98', 'NeighborOrientation': 'NeighborOrientation<0.98'}


In [7]:
def calcMeanOfLargestHistBin(arr, bins=10):
    hist, bin_edges = np.histogram(arr, bins=bins)
    idxmax = np.argmax(hist)
    binvals = arr[np.where(np.logical_and(arr>=bin_edges[idxmax], arr<bin_edges[idxmax+1]))]
    return np.mean(binvals)

def findIndexOfFirstFlat(arr, gradients=None, start_pos=0, thres=None):
    if gradients is None:
        gradients = np.gradient(arr, edge_order=2)
    assert(len(arr) == len(gradients))
    absGradients = np.abs(gradients)
    if thres is None:
        thres = calcMeanOfLargestHistBin(absGradients)
    for ix in range(start_pos, len(gradients)):
        if absGradients[ix] < thres:
            print(thres, ix)
            return ix
    return start_pos

def findIndexOfFirstZeroCrossing(arr, gradients=None, start_pos=0):
    if gradients is None:
        gradients = np.gradient(arr, edge_order=2)
    assert(len(arr) == len(gradients))
    start = start_pos+1 if start_pos == 0 else start_pos
    for ix in range(start_pos, len(gradients)):
        if gradients[ix]*gradients[ix-1] <=0 :
            return ix
    return start_pos


flt = [0.10650698, 0.78698604, 0.10650698] # sigma = 0.5
#flt = [0.17, 0.66, 0.17] # sigma 0.6
#flt = [0.25, 0.5, 0.25] # sigma 0.8
print(flt)
def smoothSignal(arr):
    padArr = np.zeros((arr.shape[0]+2,) )
    padArr[0] = arr[0]
    padArr[1:-1] = arr
    padArr[-1] = arr[-1]
    '''
    arr = list(arr)
    padArr = arr[0:1] + arr[:] + arr[-1:]
    '''
    newArr = np.convolve(padArr, flt, 'valid')
    if len(arr) != len(newArr):
        raise ValueError("inequal length {} {}".format(len(arr), len(newArr)))
    return newArr

[0.10650698, 0.78698604, 0.10650698]


In [15]:
def categorizeFilters(filters):
    if filters is None:
        newfilters = {}
    elif not isinstance(filters, dict):
        newfilters = {}
        for col in allNeighborColNames:
            for strFlt in filters:
                if col in strFlt:
                    newfilters[col] = strFlt
                    break
    filters = newfilters
    # print(filters)
    return filters

def applyNeighborRuleModelPerVLine(linedf, filters, maxTailLenth=20, smooth=True):
    dominant_issues = []
    linedf.loc[:, 'ClfLabel'] = 1

    # step 1, search and apply from head
    headdf = linedf.loc[linedf.index[:maxTailLenth], :]
    minNeighborOrientation, minNeighborParalism = headdf.min()[allNeighborColNames[1:]]
    issue_feature, issue_index = None, None
    if minNeighborParalism < minNeighborOrientation and len(headdf.query(filters['NeighborParalism'])) > 0:
        issue_feature = 'NeighborParalism'
        issue_index = np.argmin(headdf[issue_feature].values)
    elif minNeighborOrientation < minNeighborParalism and len(headdf.query(filters['NeighborOrientation'])) > 0:
        issue_feature = 'NeighborOrientation'
        issue_index = np.argmin(headdf[issue_feature].values)
    dominant_issues.append(None)
    if issue_feature is not None:
        dominant_issues[0] = [issue_feature, issue_index]
        arr = linedf[issue_feature].values
        if smooth:
            arr = smoothSignal(arr)
        gradient = np.gradient(arr, edge_order=2)
        idxFlat = findIndexOfFirstFlat(arr, gradient, start_pos=issue_index+1)
        dominant_issues[0].append(idxFlat)
        idxFlat = min(maxTailLenth, idxFlat)
        linedf.loc[linedf.index[:idxFlat], 'ClfLabel'] = 0

    # step 2, search and apply from tail, reverse order
    dominant_issues.append(None)
    head_index = 0 if issue_index is None else issue_index
    tailrange = len(linedf) - (head_index + 1) # exclude the head issue index itself
    if tailrange > maxTailLenth:
        taildf = linedf.loc[linedf.index[-maxTailLenth:], :]
        minNeighborOrientation, minNeighborParalism = taildf.min()[allNeighborColNames[1:]]
        issue_feature, issue_index = None, None
        if minNeighborParalism < minNeighborOrientation and len(taildf.query(filters['NeighborParalism'])) > 0:
            issue_feature = 'NeighborParalism'
            issue_index = np.argmin(taildf[issue_feature].values)
            issue_index = maxTailLenth - 1 - issue_index  # use index start from tail
        elif minNeighborOrientation < minNeighborParalism and len(taildf.query(filters['NeighborOrientation'])) > 0:
            issue_feature = 'NeighborOrientation'
            issue_index = np.argmin(taildf[issue_feature].values)
            issue_index = maxTailLenth - 1 - issue_index
        if issue_feature is not None and issue_index:
            dominant_issues[1] = [issue_feature, issue_index]
            arr = linedf[issue_feature].values[::-1]
            if smooth:
                arr = smoothSignal(arr)
            gradient = np.gradient(arr, edge_order=2)
            idxFlat = findIndexOfFirstFlat(arr, gradient, start_pos=issue_index+1)
            dominant_issues[1].append(idxFlat)
            idxFlat = min(maxTailLenth, idxFlat)
            linedf.loc[linedf.index[-idxFlat:], 'ClfLabel'] = 0
    return linedf, dominant_issues

def applyNeighborRuleModel(contourdf, filters, smooth=True):
    '''
    The step to find rule model in python:
    1. apply combined filters to find ill contour Vline candidates
    2. find the Vline candidates have dominant issues in its head+20 or tail-20
    3. remove contour issue head/tail by following rules(default is 3.1):
        * 3.1: search start from dominant issue position, new head=Index[1st flat gradient point]
        * 3.2: Index[dominant issue position], new head=Index[the gradient zero-crossing point]
    '''
    maxTailLenth = 20
    contourdf.loc[:, 'ClfLabel'] = 1
    filters = categorizeFilters(filters)

    inflection_df = contourdf.query('or '.join(filters.values()))
    # print(inflection_df[['polygonId', 'offsetx', 'offsety'] + allNeighborColNames[1:]])
    polygonIds = inflection_df.loc[:, 'polygonId'].drop_duplicates().values
    for polygonId in polygonIds:
        lineFlt = contourdf['polygonId']==polygonId
        linedf = contourdf.loc[lineFlt, :]
        newlinedf, dominant_issues = applyNeighborRuleModelPerVLine(linedf, filters, maxTailLenth, smooth)
        print(int(polygonId), dominant_issues)
        contourdf.loc[lineFlt, :] = newlinedf
    return contourdf

df = applyNeighborRuleModel(df, ['NeighborOrientation<0.98', 'NeighborParalism<0.98'])
ca.df = df
#plot_col_filter(ca, patternid='461', colname='ClfLabel')
plot_col_by_label(ca, patternid=patternid, colname='ClfLabel')

(7.6928015933137994e-05, 10)
(7.3340165057348593e-05, 13)
(14, [['NeighborOrientation', 6, 10], ['NeighborParalism', 1, 13]])
(0.00013276157630110971, 7)
(16, [['NeighborParalism', 1, 7], None])
(27, [None, None])
(28, [None, None])
(0.00021663086250959125, 7)
(29, [None, ['NeighborParalism', 2, 7]])
(0.00066472856407347853, 5)
(31, [['NeighborParalism', 1, 5], None])
[1 0]
